In [ ]:
# main.py (excerpt)

import os, sys, json

sys.path.append("/home/ul/ul_student/ul_cep22/my_folders/drl_projekt")

import gymnasium as gym
import gymnasium_robotics
import mujoco
import numpy as np

from utils.change_shape import get_geom_info, apply_override  # <-- NEW

os.environ['MUJOCO_GL'] = 'egl'
os.environ['PYOPENGL_PLATFORM'] = 'egl'

config = json.load(open("/home/ul/ul_student/ul_cep22/my_folders/drl_projekt/configs/test.json"))

gym.register_envs(gymnasium_robotics)
env = gym.make(config["env_name"], render_mode="rgb_array")
env.reset()

# Log initial object geom info (by name, not index)
obj_info_init = get_geom_info(env, "object0")
config["object_type_initial"] = obj_info_init["type"]
config["object_size_initial"] = obj_info_init["size"].tolist()
config["object_gid"] = obj_info_init["gid"]

# === DYNAMIC OVERRIDE FROM CONFIG ===
# Example in configs/test.json:
# "override_object": {"name":"object0","type":"ellipsoid","size":[0.025,0.02,0.015],"mass":2.0,"update_inertia":true}
ovr = config.get("override_object")
if ovr:
    result = apply_override(
        env,
        name=ovr.get("name", "object0"),
        typ=ovr["type"],
        size=ovr["size"],
        mass=ovr.get("mass", None),
        update_inertia=bool(ovr.get("update_inertia", False)),
        atol=1e-9,
    )
    print("Geom override:", result["ok"], result["after"])
    config["object_type_final"] = result["after"]["type"]
    config["object_size_final"] = np.asarray(result["after"]["size"]).tolist()
    config["object_inertia_updated"] = result["inertia_updated"]


Geom override: True {'gid': 23, 'type': 'ellipsoid', 'size': array([0.025, 0.02 , 0.015]), 'dataid': -1, 'body_id': 32}
